In [1]:
import json 
from pprint import pprint
from llm_handler import LLMHandler
from config import FILE_PATHS
from dotenv import load_dotenv

load_dotenv("../.env")

True

In [2]:
domain = "telecom"
category = "adaptive_tool_use"

persona_path = FILE_PATHS["personas"].format(domain=domain)
scenario_path = FILE_PATHS["scenarios"].format(domain=domain, category=category)
tool_path = FILE_PATHS["tools"].format(domain=domain)

In [3]:
model_name = "gpt-4o-mini"
# model_name = "claude-3-7-sonnet-20250219"
# model_name = "gpt-4.1-2025-04-14"
# model_name = "gemini-2.5-pro"
# model_name = "gemini-2.0-flash-001"
# model_name = "claude-sonnet-4-20250514"
# model_name = "mistral-small-2506"
# model_name = "magistral-medium-2506"
# model_name = "Qwen/Qwen3-235B-A22B-fp8-tput"
model_name = "palmyra-x5"
# model_name = "deepseek-ai/DeepSeek-R1-0528-tput"
# model_name = "Qwen/Qwen3-235B-A22B-fp8-tput"
# model_name = "amazon.nova-pro-v1:0"
# model_name = "amazon.nova-lite-v1:0"
# model_name = "deepseek-ai/DeepSeek-V3"
# model_name = "Qwen/Qwen2.5-72B-Instruct-Turbo"
# model_name = "Qwen/Qwen3-235B-A22B-fp8-tput"
# model_name = "arcee_ai/arcee-spotlight"
# model_name = "arcee-ai/caller"
# model_name = "arcee-ai/AFM-4.5B-Preview"
# model_name = "ibm-granite/granite-3.3-8b-instruct"

llm = LLMHandler().get_llm(model_name=model_name, temperature=0.0)

with open(scenario_path, "r") as f:
    scenarios = json.load(f)
    
with open(tool_path, "r") as f:
    tools = json.load(f)

#delete response_schema key from tool and keep rest of the keys
for tool in tools:
    tool.pop("response_schema", None)

llm = llm.bind_tools(tools)

In [4]:
# call the llm with the tools
system_prompt = """
You are a helpful assistant that can use the tools given to help the user. You can use multiple tools to answer the user's query.
"""

query = scenarios[0]["first_message"] 
query = "What is the balance of my account with the id 123"
# query = "i want to check all the credit transactions for my account with the id 1234567890"
# query = "I am earning 1000000 per year. I need a home loan of 100000 for 1 year"
# query = "I want to check my health score"
pprint(query)

messages = [
    {"role": "system", "content": system_prompt},
    {"role": "user", "content": query},
]

response = llm.invoke(messages)

model_name, response.content, response.tool_calls, response

'What is the balance of my account with the id 123'


BadRequestError: Error code: 400 - {'id': 'o15s5XW-2kFHot-9576d57adafb84b3', 'error': {'message': 'Unable to access non-serverless model google/gemma-3-12b-it. Please visit https://api.together.ai/models/google/gemma-3-12b-it to create and start a new dedicated endpoint for the model.', 'type': 'invalid_request_error', 'param': None, 'code': 'model_not_available'}}

In [21]:
model_name, response.usage_metadata

('Qwen/Qwen3-235B-A22B-fp8-tput',
 {'input_tokens': 6072,
  'output_tokens': 135,
  'total_tokens': 6207,
  'input_token_details': {},
  'output_token_details': {}})

In [18]:
LLMHandler.get_token_usage_info(response)

KeyError: 'reasoning'